# Part 1

In [37]:
# solve all provided instances

def solve_instances(instances):
    solutions = []
    for instance in instances:
        solutions.append(part_1_dp(instance))
    return solutions

In [38]:
test_instance = [
    {'demand': 10,
    'holdingCost': 14.1,
    'maxUnitsOrdered': 40,
    'orderingCost': 17.4,
    'stock': 0,
    'storageCapacity': 40},
    {'demand': 20,
    'holdingCost': 11.2,
    'maxUnitsOrdered': 40,
    'orderingCost': 19.2,
    'stock': 'TBD',
    'storageCapacity': 40},
    {'demand': 13,
    'holdingCost': 9.2,
    'maxUnitsOrdered': 40,
    'orderingCost': 16.4,
    'stock': 'TBD',
    'storageCapacity': 40},
    {'demand': 13,
    'holdingCost': 14.4,
    'maxUnitsOrdered': 40,
    'orderingCost': 10.4,
    'stock': 'TBD',
    'storageCapacity': 40},
    {'demand': 2,
    'holdingCost': 7.4,
    'maxUnitsOrdered': 40,
    'orderingCost': 18.1,
    'stock': 'TBD',
    'storageCapacity': 40},
    {'demand': 14,
    'holdingCost': 5.1,
    'maxUnitsOrdered': 40,
    'orderingCost': 12.3,
    'stock': 'TBD',
    'storageCapacity': 40},
    {'demand': 3,
    'holdingCost': 10.9,
    'maxUnitsOrdered': 40,
    'orderingCost': 14.2,
    'stock': 'TBD',
    'storageCapacity': 40},
    {'demand': 14,
    'holdingCost': 13.3,
    'maxUnitsOrdered': 40,
    'orderingCost': 16.3,
    'stock': 'TBD',
    'storageCapacity': 40},
    {'demand': 17,
    'holdingCost': 13.1,
    'maxUnitsOrdered': 40,
    'orderingCost': 14.0,
    'stock': 'TBD',
    'storageCapacity': 40},
    {'demand': 3,
    'holdingCost': 8.1,
    'maxUnitsOrdered': 40,
    'orderingCost': 11.6,
    'stock': 'TBD',
    'storageCapacity': 40}]

In [39]:
test_instance[0]

{'demand': 10,
 'holdingCost': 14.1,
 'maxUnitsOrdered': 40,
 'orderingCost': 17.4,
 'stock': 0,
 'storageCapacity': 40}

In [40]:
import collections
import functools

class memoized(object):
   '''Decorator. Caches a function's return value each time it is called.
   If called later with the same arguments, the cached value is returned
   (not reevaluated).
   '''
   def __init__(self, func):
      self.func = func
      self.cache = {}
   def __call__(self, *args):
      if not isinstance(args, collections.Hashable):
         # uncacheable. a list, for instance.
         # better to not cache than blow up.
         return self.func(*args)
      if args in self.cache:
         return self.cache[args]
      else:
         value = self.func(*args)
         self.cache[args] = value
         return value
   def __repr__(self):
      '''Return the function's docstring.'''
      return self.func.__doc__
   def __get__(self, obj, objtype):
      '''Support instance methods.'''
      return functools.partial(self.__call__, obj)

In [105]:
def part_1_dp_wrapper(instance):
    instance = instance

    @memoized
    def part_1_dp(week, startingInventory):
        thisWeek = instance[week]

        # boundary condition at week t = T
        if (week == len(instance)-1):
            
            # no inventory left over, therefore only one possible decision for this starting inventory
            x = thisWeek['demand'] - startingInventory
            
            # check feasibility of problem before proceeding
            if (x > thisWeek['maxUnitsOrdered'] or x < 0):
                return False

            # ordering cost
            orderingCost = thisWeek['orderingCost'] * x

            # ordering cost is only cost, therefore return
            return (orderingCost, [x])

        # recursive condition
        else:
            # dictionary with keys = costs of path, values = path
            # identical costing paths are ignored since min() will arbitrarily select a random optimal cost
            #   if there are multiple costs that are all the minimum
            possibleCostsEnum = {}
            xLowerBound = thisWeek['demand']-startingInventory
            xUpperBound = min((thisWeek['storageCapacity']+thisWeek['demand']-startingInventory), thisWeek['maxUnitsOrdered'])

            # enumerate possible decisions (order quantity x) given starting state
            for x in range(xLowerBound, xUpperBound+1):
                # ordering cost
                orderingCost = thisWeek['orderingCost'] * x

                # holding cost
                holdingCost = thisWeek['holdingCost'] * (startingInventory + x - thisWeek['demand'])

                # ending inventory
                endingInventory = x + startingInventory - thisWeek['demand']
                
                # check if current path is feasible
                call_to_future = part_1_dp(week+1, endingInventory)
                
                if not call_to_future:
                    # problem is infeasible in current state, ignore this enumerated possibility and check next
                    continue
                else:
                    # minimum cost from time t+1 for this particular enum 
                    # unpack tuple of shape (cost, [path])
                    minimumFutureCost, optimalFuturePath = call_to_future
                
                    # calculate total cost from this starting state
                    cost = orderingCost + holdingCost + minimumFutureCost

                    # append current ordering quantity (x) to optimalFuturePath
                    newPath = [x] + optimalFuturePath

                    # insert into dict if cost does not exist yet in possibleCostsEnum dict
                    if cost not in possibleCostsEnum:
                        possibleCostsEnum[cost] = newPath
                        
             # check if this starting state results in any feasible solutions
            if len(possibleCostsEnum.keys()) == 0:
                return False
            else:
                # check for min key, since keys are all costs of paths
                minCost = min(possibleCostsEnum.keys())

                # return tuple of shape (cost, [path])
                return (minCost, possibleCostsEnum[minCost])
        
    return part_1_dp(0, 0)

In [106]:
# test run
print(part_1_dp_wrapper(test_instance))

(1658.4, [10, 20, 13, 13, 2, 14, 3, 14, 17, 3])


# Part 2

In [42]:
part2_test_instance = [{'demand': 10,
  'holdingCost': 14.1,
  'maxUnitsOrdered': 40,
  'orderingCost': 17.4,
  'shortageCost': 33.4,
  'stock': 0,
  'storageCapacity': 40},
 {'demand': 13,
  'holdingCost': 15.0,
  'maxUnitsOrdered': 40,
  'orderingCost': 16.2,
  'shortageCost': 23.4,
  'stock': 'TBD',
  'storageCapacity': 40},
 {'demand': 19,
  'holdingCost': 5.4,
  'maxUnitsOrdered': 40,
  'orderingCost': 16.7,
  'shortageCost': 28.4,
  'stock': 'TBD',
  'storageCapacity': 40},
 {'demand': 2,
  'holdingCost': 7.4,
  'maxUnitsOrdered': 40,
  'orderingCost': 18.1,
  'shortageCost': 19.6,
  'stock': 'TBD',
  'storageCapacity': 40},
 {'demand': 8,
  'holdingCost': 12.2,
  'maxUnitsOrdered': 40,
  'orderingCost': 10.1,
  'shortageCost': 26.9,
  'stock': 'TBD',
  'storageCapacity': 40},
 {'demand': 17,
  'holdingCost': 11.3,
  'maxUnitsOrdered': 40,
  'orderingCost': 11.4,
  'shortageCost': 28.7,
  'stock': 'TBD',
  'storageCapacity': 40},
 {'demand': 17,
  'holdingCost': 13.1,
  'maxUnitsOrdered': 40,
  'orderingCost': 14.0,
  'shortageCost': 18.2,
  'stock': 'TBD',
  'storageCapacity': 40},
 {'demand': 2,
  'holdingCost': 6.3,
  'maxUnitsOrdered': 40,
  'orderingCost': 13.1,
  'shortageCost': 33.8,
  'stock': 'TBD',
  'storageCapacity': 40},
 {'demand': 10,
  'holdingCost': 12.9,
  'maxUnitsOrdered': 40,
  'orderingCost': 10.3,
  'shortageCost': 16.3,
  'stock': 'TBD',
  'storageCapacity': 40},
 {'demand': 16,
  'holdingCost': 11.9,
  'maxUnitsOrdered': 40,
  'orderingCost': 15.3,
  'shortageCost': 24.3,
  'stock': 'TBD',
  'storageCapacity': 40}]

In [107]:
def part_2_dp_wrapper(instance):
    instance = instance

    @memoized
    def part_2_dp(week, startingInventory):
        thisWeek = instance[week]

        # boundary condition at week t = T
        if (week == len(instance)-1):
            # dictionary with keys = costs of path, values = path
            # identical costing paths are ignored since min() will arbitrarily select a random optimal cost
            #   if there are multiple costs that are all the minimum
            possibleCostsEnum = {}
            
            # assumed requirement: no inventory left over, therefore only explore x = 0 to x = demand - inventory
            xLowerBound = 0
            xUpperBound = thisWeek['demand'] - startingInventory

            # enumerate possible decision (order quantity x) given starting inventory
            for x in range(xLowerBound, xUpperBound + 1):
                
                # check feasibility of problem before proceeding
                if (x > thisWeek['maxUnitsOrdered'] or x < 0):
                    return False
                
                # unsatisfied demand
                shortage = thisWeek['demand'] - startingInventory - x
                
                # don't consider holding cost as inventory needs to be 0 at the end
                
                # ordering cost
                orderingCost = thisWeek['orderingCost'] * x
                
                # shortage cost
                shortageCost = thisWeek['shortageCost'] * shortage
                
                cost = orderingCost + shortageCost
                
                # insert into dict if cost does not exist yet in possibleCostsEnum dict
                if cost not in possibleCostsEnum:
                    possibleCostsEnum[cost] = [x]
                    
            # check if this starting state results in any feasible solutions
            if len(possibleCostsEnum.keys()) == 0:
                return False
            else:
                # check for min key, since keys are all costs of paths
                minCost = min(possibleCostsEnum.keys())
                
                # return tuple of shape (cost, [path])
                return (minCost, possibleCostsEnum[minCost])
            
        # recursive condition
        else:
            # dictionary with keys = costs of path, values = path
            # identical costing paths are ignored since min() will arbitrarily select a random optimal cost
            #   if there are multiple costs that are all the minimum
            possibleCostsEnum = {}
            xLowerBound = 0
            xUpperBound = min((thisWeek['storageCapacity']+thisWeek['demand']-startingInventory), thisWeek['maxUnitsOrdered'])

            # enumerate possible decisions (order quantity x) given starting state
            for x in range(xLowerBound, xUpperBound+1):
                # unsatisfied demand, account for positive ending inventory 
                shortage = max(0, thisWeek['demand'] - startingInventory - x)
                
                # ordering cost
                orderingCost = thisWeek['orderingCost'] * x

                # holding cost
                holdingCost = thisWeek['holdingCost'] * (startingInventory + x - thisWeek['demand'])

                # ending inventory, account for positive shortage calculation causing ending inventory to become negative
                endingInventory = max(0, x + startingInventory - thisWeek['demand'])
                
                # shortage cost
                shortageCost = thisWeek['shortageCost'] * shortage
                
                # check if current path is feasible
                call_to_future = part_2_dp(week+1, endingInventory)
                
                if not call_to_future:
                    # problem is infeasible in current state, ignore this enumerated possibility and check next
                    continue
                else:
                    # minimum cost from time t+1 for this particular enum 
                    # unpack tuple of shape (cost, [path])
                    minimumFutureCost, optimalFuturePath = call_to_future

                    # calculate total cost from this starting state
                    cost = orderingCost + holdingCost + shortageCost + minimumFutureCost

                    # append current ordering quantity (x) to optimalFuturePath
                    newPath = [x] + optimalFuturePath

                    # insert into dict if cost does not exist yet in possibleCostsEnum dict
                    if cost not in possibleCostsEnum:
                        possibleCostsEnum[cost] = newPath
            
            # check if this starting state results in any feasible solutions
            if len(possibleCostsEnum.keys()) == 0:
                return False
            else:
                # check for min key, since keys are all costs of paths
                minCost = min(possibleCostsEnum.keys())

                # return tuple of shape (cost, [path])
                return (minCost, possibleCostsEnum[minCost])
        
    return part_2_dp(0, 0)

In [108]:
# test run
print(part_2_dp_wrapper(part2_test_instance))

(1291.2, [10, 0, 19, 0, 8, 17, 0, 2, 0, 16])
